# Data converter from a CSM data file to YM2203 f_number and block data file.

In [1]:
import os
import sys
import math

import numpy as np

## Read CSM file

Time resolution is 10ms. Each line contains freq and amp data for 10ms period.    

**Note**: CSM file line format  
```
freq1,amp1,freq2,amp2,freq3,amp3,freq4,amp4
```

In [2]:
input_file = './csm_enc_synth/resources/apollo11_launch.csm'
with open(input_file, 'r') as f:
    lines = [[float(dt) for dt in (l.rstrip('\n')).split(',')] for l in f.readlines()]

# CSM data = [ [[f1,a1],[f2,a2],[f3,a3][f4,a4]], [[f1,a1],[f2,a2]].... ]
csm_data = []
for line in lines:
    item = []
    for idx in range(4):
        item.append([line[idx*2], line[idx*2+1]])
    csm_data.append(item)

print(len(csm_data) * 0.01, 'sec')

25.26 sec


## Convert (freq,amp) to (f_num, blk, amp) format

```
input = f1,a2,f2,a2,f3,a3,f4,a4
output = fnum1,blk1,amp1,fnum2,blk2,amp2,fnum3,blk3,amp3,fnum4,blk4,amp4
```

In [3]:
#def freq_to_fnum(freq, prescaler):
#    block = 1
#    while True:
#        f_num = (freq * (1<<20) / (1.2288e6/(12*prescaler))) / (1<<(block-1))
#        f_num = int(f_num)
#        if f_num <= 0x3ff:
#            break
#        block += 1
#    return f_num, block

# Convert frequency to F-number and block data for FM-sound device.
def freq_to_fnum(freq, prescaler):
    block = 1
    f_num = ((freq * (1<<20)) / (1.2288e6/(12*prescaler))) / (1<<(block-1)) # prescaler=[2|3|6]
    f_num = int(f_num)
    if f_num > 0:  
        bits = int(math.log10(f_num) / math.log10(2))   # how many bits are required to express f_num in binary?
        if bits >= 10:                                  # 2^10 = 0x400
            block = bits - 10 + 2
            f_num >>= block - 1
    return f_num, block

# Convert amplitude to total-level data for FM-sound device (dB)
# 1 bit = -0.75dB (YM2203), total 7 bits (0x7f == -95.25dB)
def amp_to_tl(amp, max_amp):
    if amp == 0:
        return 0x7f
    db = -20 * math.log10(amp / max_amp)
    total_level = int(db / 0.75)
    return total_level


# Find the max amplitude value from the CSM data for dB calculation
amp_max = 0
amp_min = 10e20
for csm in csm_data:
    for c in range(4):
        _, amp = csm[c]
        amp_max = max(amp_max, amp)
        amp_min = min(amp_min, amp)
print(f'amplitude min:{amp_min}, max:{amp_max}')

base, _ = os.path.splitext(input_file)
output_file = base+'_ym2203.txt'

# Generate data for YM2203 CSM playback 
# format: fnum1,blk1,tl1,fnum2,blk2,tl2,...,blk4,tl4
with open(output_file, 'w') as f:
    ym2203_csm = []           # ym2203_csm holds CMS data to generate a 'Mutsu' comatible register access log file in the next code block section
    prescaler = 2
    for csm in csm_data:
        ym2203_csm.append([])
        for c in range(4):
            freq, amp = csm[c]
            fnum, blk = freq_to_fnum(freq, prescaler)
            tl = amp_to_tl(amp, amp_max)
            if c != 0:
                print(',', end='', file=f)
            print(f'{fnum},{blk},{tl}', end='', file=f)
            ym2203_csm[-1].append(fnum)
            ym2203_csm[-1].append(blk)
            ym2203_csm[-1].append(tl)
        print(file=f)

print(output_file, 'is generated.')


amplitude min:0.0, max:17.74576759338379
./csm_enc_synth/resources/apollo11_launch_ym2203.txt is generated.


In [4]:
def ym2203_reg(reg, val, timestamp=0, f=sys.stdout):
    print(f'YM2203C Reg[${reg:02x}]=${val:02x} at {timestamp}', file=f)

# Generate 'Mutsu', an FM-7 series emulator compatible register access log data.
timestamp = 0
base, _ = os.path.splitext(input_file)
output_file = base+'_csm.log'
with open(output_file, 'w') as f:
    ym2203_reg(0x2f, 0x00, timestamp, f)            # set prescaler to 2
    reg_table = [[0xa9, 0xad, 0x42], [0xaa, 0xae, 0x46], [0xa8, 0xac, 0x4a], [0xa2, 0xa6, 0x4e]]
    for dt in ym2203_csm:
        for c in range(4):
            fn = dt[c * 3    ]
            bl = dt[c * 3 + 1]
            tl = dt[c * 3 + 2]
            regs = reg_table[c]
            fn1 = fn & 0xff
            fn2 = (bl<<3) | ((fn >> 8) & 0x07)
            ym2203_reg(regs[0], fn1, timestamp, f)
            ym2203_reg(regs[1], fn2, timestamp, f)
            ym2203_reg(regs[2], tl, timestamp, f)
            ym2203_reg(0x25, 0x54, timestamp, f)
        ym2203_reg(0x24, 0x3c, timestamp, f)
        timestamp += 10e6

print(output_file, 'is generated.')

./csm_enc_synth/resources/apollo11_launch_csm.log is generated.
